# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [28]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,214.00 (365 días después).


Pregunta 2:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 3:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 3282 y 4454 días.


Pregunta 4:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 5:

Considere un swap a 1Y con cupones semestrales de 1.9% (lineal). Si la tasa
cupón cero a 0.5Y es 1.8% (exp), calcule la tasa cero a 1Y.


Pregunta 6:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.1%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 7:

Calcule un paso de simulac

## Respuestas

### Pregunta 1

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,214.00 (365 días después).

RESPUESTA: 
    
Se debe utilizar las siguientes ecuaciones: 
    
$$ICP_{t+d}=Round \left( ICP_{t}\cdot\left(1+r_{t}\cdot \frac{d}{360} \right),2 \right)$$

Siendo d el número de días entre t la próxima fecha hábil para la ecuacion del ICP y T para el TNA.

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

In [74]:
ICP0=10000
ICP365=10214

TNA = round((10214.0 / 10000.0 - 1) * (360.0 / 365), 4)

print(f'{TNA:.4%}')

2.1100%


### Pregunta 2

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el modelo de Hull-White en la medida T-forward.

RESPUESTA: 

La diferencia esta en que bajo la medida forward y la medida libre se riesgo esta en que bajo la primera medida se puede sacar el valor del factor de descuento de la esperanza, no teniendo así que "devolverme por la trayectoria" para obtener el valor presente del payoff. 

Devolverse por la trayectoria implica calcular para cada salto en el tiempo (dt) de cada simulación la multimplicación entre el factor de descuento por el flujo en el tiempo t y luego sumarlo para obtener el valor presente de cada simulación. Finalmente obteniendo el valor final calculando el promedio de las simulaciones. 

No devolverse por la tractoria es que calculo el payoff esperado calculando el promedio de las simulaciones.


MEDIDA FORWARD

\begin{equation}
V\left(r_t,t,T\right)=Z\left(r_t,t,T\right)\mathbb{E}_t^{Z_T}\left[g\left(r_T,T\right)\right]
\end{equation}

MEDIDA LIBRE DE RIESGO

\begin{equation}
V\left(r_t,t\right)=\mathbb{E}_t^*\left[e^{-\int_{t}^{T}R\left(r_u\right)du}g\left(r_T,T\right)\right] 
\end{equation}


### Pregunta 3

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa forward entre 3282 y 4454 días.
RESPUESTA: 

In [68]:
t1=3282
t2=4454

r1=curva(t1)
r2=curva(t2)

df1=math.exp(-r1*t1/365)
df2=math.exp(-r2*t2/365)

fwd=(df1/df2-1)*365/(t2-t1)

print(f'{fwd:.8%}')

1.25210516%


TASA FORWARD

\begin{equation}
\mathbb{E}_t^Q\left(L_{T_f}\right)=\left(\frac{P_L\left(t,T_f\right)}{P_L\left(t,T_{f+3M}\right)}-1\right)\cdot\frac{1}{yf\left(T_f,T_{f+3M}\right)}=FWD\left(T_f,T_{f+3M}\right)
\end{equation}

### Pregunta 4

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es la misma que en el modelo de Hull-White. Comente.

RESPUESTA:
    
Si la ecución es la misma, esto es posible dado a que la ecuación de black y scholes se puede utilizar mientras que
el subyacente tenga una distribución log-normal. Esto es el caso de Vasicek y Hull-White dado que la ecuación 
de la tasa instantanea varía en los modelos esta distribuye normal, por lo que al aplicar la ecuación siguiente ecuacion para obtener el bono cupon cero: 

\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right] \\
\end{equation}

Nos damos cuenta que este distribuye lognormal haciendo posible lo anterios, a pesar que A y B no sean las mismas funciones. 


### Pregunta 5

Considere un swap a 1Y con cupones semestrales de 1.9% (lineal). Si la tasa cupón cero a 0.5Y es 1.8% (exp), 
calcule la tasa cero a 1Y.

RESPUESTA:

La ecuaciones a utilizar son para el factor de descuento del primer cupon: 

$$
\begin{equation}
\Pi^{fija,tenor}\left(T_0\right)=P_c\left(T_0,T_{tenor}\right)\cdot\left(1+r\cdot\frac{T_{tenor}-T_0}{360}\right)=1
\end{equation}
$$

Para el factor de descuento del segundo cupón:

$$
\begin{equation}
\Pi^{fija}\left(T_0\right)=P_c\left(t_0,T_1\right)\cdot r\cdot\frac{T_1-T_0}{360}+P_c\left(t_0,T_2\right)\left(1+r \cdot \frac{T_2-T_1}{360}\right)=1
\end{equation}
$$

Pues en esta se pagan dos cupones y se debe utilizar el factor de descuento del primer cupon obtenido con la tasa cupón cero 
para obtener el segundo. 
Cabe mencionar que se considerara nocional igual a 1 y siendo r =1.9%


In [75]:
Fd_c1 = math.exp(-1.8/100*180/365) #Payoff primer cupon

Fd_c2 = (1-Fd_c1*0.019*180/360)/(1+0.019*180/360)

tasa_1Y= -math.log(Fd_c2)*365/180 

# AD: Fd_c2 = (1-Fd_c1*0.019*180/360)/(1+0.019*180/360) = exp(-tasa_1Y * 365/360)
# tasa_1Y = -log(Fd_c2) * 360 / 365

tasa_1Y

0.03835706806349034

### Pregunta 6

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5% valorice una call a 1Y sobre un bono 
cero cupón a 2Y con un strike de 99.1%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

RESPUESTA:

Recordar que el orden de los parametros de la fución son:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
Cabe mencionar, que los parametros gamma y sigma del enunciado coinciden con los de la seccion 16.4.
    

In [63]:
strikes = [.991]
for k in strikes:
    V = hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),   #1 año
        math.exp(-curva(730) * 730 / 365.0),  #2 años
        gamma,
        sigma
    )

print(f'{V:.8}')

0.0085612992


### Pregunta 7

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando gamma = 1.0, sigma = 0.5%, dt = 1/264, r0
obtenido de la curva cupón cero entregada y número aleatorio N(0, 1) igual a .5504 .

RESPUESTA:

In [1]:
# AD: pucha ....